In [4]:
# reading DARDAR .h5 files and outputting monthly files for totals 
#  and counts onto a chosen grid, which can then be read by another
#  python script to create a multi-month mean grid
# dduncan, 14/12/17
# updated 2/1/18 for doing day and night separately (or both in future, i guess)

import h5py
from mpl_toolkits.basemap import Basemap, cm
from pyhdf.SD import SD, SDC

In [5]:

dir = '/home/dudavid/Dendrite/Dendrite/SatData/DARDAR/'
#yrs = ['08','09'] # day and night data years on dendrite
yrs = ['15'] # day and night data years on dendrite
dn = 'day'
startlat = 90.0 # abs value of NS boundaries
gsize = 2.5 # grid size in degree
nx, ny = int(360/gsize), int(startlat*2/gsize)

# try doing 6 pixel long average of DARDAR data
nsmo = 3 # double this so 6? -- [ x-nsmo : n+nsmo ]

In [ ]:
for yr in yrs:
    #yr = '09'
    print('starting year 20'+yr)
    dirt = dir+'20'+yr+'/'
    months = sorted(os.listdir(dirt))
    print(months)
    for m in months[:]:
        #if os.path.isfile(outfi):
        #    print(outfi,' already exists!')
        #else:
        print('month: ',m)
        iwpg,iwpgc = np.zeros([ny,nx]), np.zeros([ny,nx]) # re-init for each month
        iwpall = []
        days = sorted(os.listdir(dirt+m+'/'))
        ndays = 0 #init
        for d in days:
            print(d)
            files = sorted(os.listdir(dirt+m+'/'+d))
            if len(files) != 0:  # in case no files in day dir
                ndays += 1
                for fix in files[:]: #range(len(files)):
                    file = dirt+m+'/'+d+'/'+fix
                    #print(file)

                    if yr == '15' or yr == '08':
                        f = h5py.File(file,'r')
                        iw = np.array(f['iwc']) # IWC in kg/m^3
                        lat = np.array(f['latitude']) #[mask]
                        lon = np.array(f['longitude']) # [mask]
                        dnf = np.array(f['day_night_flag']) # night=1, day=0
                    else:
                        f = SD(file,SDC.READ)
                        iw = f.select('iwc')[:,:]
                        lat = f.select('latitude')[:]
                        lon = f.select('longitude')[:]
                        dnf = f.select('day_night_flag')[:] # night=1, day=0
                    #print(np.shape(iw),np.shape(lat))
                    #flag = np.array(f['land_water_mask'])
    # mask is from calipso flags where:
    # (http://www.icare.univ-lille1.fr/projects_data/dardar/docs/varcloud-algorithm_description-v1.0.pdf)
    #  0=shallow ocean 1=land 2=coastlines 3=shallow inland water 4=intermittent water 
    #  5=deep inland water 6=coastal ocean 7=deep ocean
                    if( m == months[0] and d == days[0] and fix == files[0] ): # don't calc every time (always the same)
                        if yr=='15' or yr=='08': hgt = np.array(f['height']) #height above sea level in m [436]
                        if yr!='15' and yr!='08': hgt = f.select('height')[:] #height above sea level in m [436]
                        hmask = []
                        for z in range(len(hgt)):
                            if hgt[z]>=0 and hgt[z]<20000:  hmask.append(z)
                    #mask = flag >= 0 # all!
                    #mask = flag > 4 # ocean and deep inland water

                    if dn == 'day': mask = np.array([dnf == 0])[0,:] # DAYTIME ONLY
                    if dn == 'night': mask = np.array([dnf == 1])[0,:] # NIGHTTIME ONLY

                    #print(np.shape(mask),np.shape(dn),np.shape(iw))
                    iwc = iw[mask,:] # just selected profiles
                    lat = lat[mask]
                    lon = lon[mask]
                    #iwc = iw[:,:] # all

                    #dmsk = np.array(f['DARMASK_Simplified_Categorization'])[mask] 
                    #as follows (same source doc):
                    # -9 ground, -1 dunno, 0 clear, 1 ice, 2 ice+supercooled, 3 liquid (warm),
                    #  4 supercooled, 5 rain, 6 aerosol, 7 maybe insects, 8 stratospheric feature
                    #dayn = np.array(f['day_night_flag'])[mask] #day=0, night=1

                    # calculate lat/lon indices for Xdeg grid (fixed 23/11):
                    ladex = np.round((lat[:]+startlat-gsize*.5) / gsize)
                    lodex = np.round((lon[:]+180.0-gsize*.5) / gsize)

                    iwp = [sum(iwc[x,hmask]*60.0) for x in range(len(iwc[:,20]))]
                    ariwp = np.array(iwp)
                    scrn = [abs(lat) <= 60]
                    ariwph= np.array(ariwp[scrn]) ##NEW! just 60NS for histo
                    # all range gates are 60m here, so kg/m3 * m yields IWP in kg/m2

                    # running mean over iwp series:
                    iwpsmoo = ariwp # first few will be same, so will last few. 
                    iwpsmooh = ariwph # first few will be same, so will last few. 
                    iwpsmoo[nsmo:(len(ariwp)-nsmo)] = \
                     [ariwp[(x-nsmo):(x+nsmo)].mean() for x in range(nsmo,(len(ariwp)-nsmo))]
                    iwpsmooh[nsmo:(len(ariwph)-nsmo)] = \
                     [ariwph[(x-nsmo):(x+nsmo)].mean() for x in range(nsmo,(len(ariwph)-nsmo))]
                    iwpall.extend(iwpsmooh[::nsmo]) # no point storing all points...

                    ### uncomment the below (and save statements below) to output monthly grids of total/counts
                    #for x in range(nx):
                    #    if np.any(lodex == x):
                    #        dexsub = np.where(lodex == x)
                    #        subset = iwpsmoo[dexsub] # using smoothed iwp or not?
                    #        subsetla = ladex[dexsub]
                    #        for y in range(ny):
                    #            if np.any(subsetla == y):
                    #                lolasubset = np.where(subsetla == y)
                    #                iwpg[y,x] += subset[lolasubset].sum() # add up, get mean afterward
                    #                iwpgc[y,x] += len(subset[lolasubset])


        nd = str(ndays).rjust(2,'0')
        # create histogram of IWP
        binz = [2**x for x in range(-14,6)]
        binzz = [-.001,2**(-14)]
        hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
        histaz, bin_edges = np.histogram(np.array(iwpall),bins=binzz)
        print('histaz',histaz,hista)
        fin = 'grids/dardar.histo.60NS.day.v2.'+yr+m+'.'+nd
        outz = np.append(histaz,hista)
        np.save(fin,outz) #hista)
        #hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
        # dataset name, res (x,y), time res of input, output arrays (grids, zonals), version?, # days
        #fin = 'grids/dardar.'+dn+'.'+str(nx)+'x.'+str(ny)+'y.iwptot.cts.hist.v1.'+yr+m+'.'+nd
        #fin = 'grids/dardar.'+str(nx)+'x.'+str(ny)+'y.iwptot.cts.hist.v1.'+yr+m+'.'+nd
        #np.save(fin,hista)
        #np.save(fin+'.tot',iwpg)
        #np.save(fin+'.cts',iwpgc)


starting year 2015
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
month:  01
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
histaz [666653] [ 7784 13582 22012 34673 48520 59018 67864 74780 86380 89602 64496 53368
 40672 29947 23818 17719  5531   308     2]
month:  02
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
histaz [541355] [ 6393 11217 18049 28522 40159 50099 57655 62318 71044 72807 52132 43543
 33605 24057 18291 13194  4771   307     2]
month:  03
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
histaz [343722] [ 3388  6026 10310 16287 23559 29664 33958 37587 42688 43315 29349 24100
 18864 14817 12335  8173  2807   174     2]
month:  04
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
histaz [648250] [ 7563 13431 22705 39056 56169 70273 78252 81771 90468 92622 63507 53809
 413